In [65]:
# Autoreload possibly interferes with IntelliJ debugging
# %reload_ext autoreload
# %autoreload 2
import logging
from datetime import datetime
# class FlushHandler(logging.StreamHandler):
#     def emit(self, record):e
#         super().emit(record)
#         self.flush()

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# log = lambda msg: logging.info(msg)
log = lambda msg: print(f"{datetime.today()} {msg}")


In [66]:
#assert False
global_force = False

# Full night pipeline (multiple files)

In [67]:
import pandas as pd
import os

input_dir = "C:\\dev\\play\\brainwave-data"
stats_df = pd.read_csv(input_dir + os.path.sep + "stats.csv")

# Find Brainflow files that haven't been copied over

In [68]:
import pytz
from tqdm import tqdm
import paramiko
import re
from datetime import datetime, timedelta
import os
import dotenv
dotenv.load_dotenv()

# Define the time window
time_window = timedelta(minutes=10)

# Define the remote server details
hostname = os.getenv('SSH_HOST')
username = os.getenv('SSH_USERNAME')
password = os.getenv('SSH_PASSWORD')
remote_dir = '/home/graham/dev/Brainwave-Data'
local_dir = input_dir

# Create an SSH client
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname, username=username, password=password, compress=True)

# List files in the remote directory
stdin, stdout, stderr = ssh.exec_command(f'ls {remote_dir}')
remote_brainflow_files = [f for f in stdout.read().decode().splitlines() if f.endswith(".brainflow.csv")]


In [69]:
remote_brainflow_files

['2024-09-30-21-27-49.brainflow.csv',
 '2024-10-01-21-36-11.brainflow.csv',
 '2024-10-02-21-21-15.brainflow.csv',
 '2024-10-04-21-13-39.brainflow.csv',
 '2024-10-05-21-09-47.brainflow.csv',
 '2024-10-06-21-29-18.brainflow.csv',
 '2024-10-07-21-35-18.brainflow.csv',
 '2024-10-08-21-21-34.brainflow.csv',
 '2024-10-09-21-22-04.brainflow.csv',
 '2024-10-11-21-16-27.brainflow.csv',
 '2024-10-15-22-01-52.brainflow.csv',
 '2024-10-21-21-30-19.brainflow.csv',
 '2024-10-23-21-24-03.brainflow.csv',
 '2024-10-24-21-35-33.brainflow.csv',
 '2024-10-29-20-42-12.brainflow.csv',
 '2024-10-30-21-26-55.brainflow.csv',
 '2024-11-01-21-32-22.brainflow.csv',
 '2024-11-02-21-29-03.brainflow.csv',
 '2024-11-04-21-20-51.brainflow.csv',
 '2024-11-05-21-34-15.brainflow.csv',
 '2024-11-07-21-26-18.brainflow.csv',
 '2024-11-08-21-16-53.brainflow.csv',
 '2024-11-09-21-27-51.brainflow.csv',
 '2024-11-12-21-11-43.brainflow.csv',
 '2024-11-16-21-54-46.brainflow.csv',
 '2024-11-19-21-29-04.brainflow.csv',
 '2024-11-20

In [70]:
files_to_copy = remote_brainflow_files.copy()

for root, dirs, files in os.walk(input_dir):
    #for idx, file_name in enumerate(tqdm(dirs, desc="Converting Brainflow to FIF")):
    for idx, file_name in enumerate(files):
        full_input_filename = os.path.join(root, file_name)
        if ".brainflow.csv.bz2" in full_input_filename:
            file_name_without_bz2 = file_name.removesuffix(".bz2")
            already_have = file_name_without_bz2 in remote_brainflow_files
            print(f"Already have {full_input_filename}: {already_have}")
            if already_have:
                files_to_copy.remove(file_name_without_bz2)



Already have C:\dev\play\brainwave-data\08-07-2024--22-51-16\08-07-2024--22-51-16.brainflow.csv.bz2: False
Already have C:\dev\play\brainwave-data\09-07-2024--22-52-25\09-07-2024--22-52-25.brainflow.csv.bz2: False
Already have C:\dev\play\brainwave-data\2024-07-15-09-10-07\2024-07-15-09-10-07.brainflow.csv.bz2: False
Already have C:\dev\play\brainwave-data\2024-07-16-23-14-52\2024-07-16-23-14-52.brainflow.csv.bz2: False
Already have C:\dev\play\brainwave-data\2024-07-24-18-57-03\2024-07-24-18-57-03.brainflow.csv.bz2: False
Already have C:\dev\play\brainwave-data\2024-09-03-21-10-58\2024-09-03-21-10-58.brainflow.csv.bz2: False
Already have C:\dev\play\brainwave-data\2024-09-04-21-44-08\2024-09-04-21-44-08.brainflow.csv.bz2: False
Already have C:\dev\play\brainwave-data\2024-09-08-21-01-39\2024-09-08-21-01-39.brainflow.csv.bz2: False
Already have C:\dev\play\brainwave-data\2024-09-10-21-22-21\2024-09-10-21-22-21.brainflow.csv.bz2: False
Already have C:\dev\play\brainwave-data\2024-09-11-

In [71]:
files_to_copy

['2025-01-11-21-19-30.brainflow.csv']

In [72]:
import os
import bz2
from tqdm import tqdm

sftp = ssh.open_sftp()
for file in files_to_copy:
    remote_file_path = remote_dir + "/" + file
    local_file_path = os.path.join(input_dir, file)
    log(f"Copying Brainflow backup remote:{remote_file_path} to {local_file_path}")

    # Get the file size
    remote_file_size = sftp.stat(remote_file_path).st_size

    with tqdm(total=remote_file_size, unit='B', unit_scale=True, desc=file, ascii=True) as pbar:
        local_file_path = os.path.join(input_dir, file)
    
        def callback(transferred_so_far, total_to_transfer):
            pbar.update(transferred_so_far - pbar.n)
    
        # Create a file-like object that writes to the local file
        with open(local_file_path, 'wb') as local_file:
            sftp.getfo(remote_file_path, local_file, callback=callback)
sftp.close()

2025-01-12 08:45:04.237565 Copying Brainflow backup remote:/home/graham/dev/Brainwave-Data/2025-01-11-21-19-30.brainflow.csv to C:\dev\play\brainwave-data\2025-01-11-21-19-30.brainflow.csv


2025-01-11-21-19-30.brainflow.csv: 100%|##########| 2.04G/2.04G [00:25<00:00, 78.7MB/s]   


In [73]:
ssh.close()


## Convert Brainflow files to FIF

In [74]:
def get_brainflow_compressed_filename(full_input_filename: str) -> str:
    full_output_dirname = webserver.output_dirname(full_input_filename)
    compressed_full_output_filename = str(os.path.join(full_output_dirname, os.path.basename(full_input_filename))) + '.bz2'
    return compressed_full_output_filename

In [75]:
from datetime import datetime

import webserver
import convert
# import zstandard as zstd
import os
import bz2
import time
import shutil

errors = []
processed = []

# Could get these working later
skip_list = ['2024-09-10-21-22-21']

def compress_bz2(input_file, output_file):
    start_time = time.time()
    with open(input_file, 'rb') as f_in:
        with bz2.open(output_file, 'wb', compresslevel=9) as f_out:
            shutil.copyfileobj(f_in, f_out)
    end_time = time.time()
    return end_time - start_time, os.path.getsize(output_file)

for root, dirs, files in os.walk(input_dir):
    #for idx, file_name in enumerate(tqdm(dirs, desc="Converting Brainflow to FIF")):
    for idx, file_name in enumerate(files):
        full_input_filename = os.path.join(root, file_name)
        try:
            if full_input_filename.endswith(".brainflow.csv"):
                full_output_dirname = webserver.output_dirname(full_input_filename)
                full_output_filename = str(os.path.join(full_output_dirname, 'raw.fif'))
                
                compressed_full_output_filename = get_brainflow_compressed_filename(full_input_filename)
                
                if not os.path.exists(compressed_full_output_filename):
                    log(f"Compressing file {full_input_filename} to " + compressed_full_output_filename)
                    processed.append("Compressing " + full_input_filename)
                    try:
                        os.mkdir(os.path.dirname(compressed_full_output_filename))
                    except:
                        pass
                    compress_bz2(full_input_filename, compressed_full_output_filename) 
                    
                if os.path.exists(full_output_filename):
                    log(f"Skipping file {full_input_filename} as {full_output_filename} and {compressed_full_output_filename} already exist")
                    continue
                should_skip = False
                for s in skip_list:
                    if s in full_input_filename:
                        log(f"Skipping file {full_input_filename}")
                        should_skip = True
                if not should_skip:
                    log(f"Processing file {full_input_filename}")
                    processed.append("Processing " + full_input_filename)
                    channels = None
                    date_time_str = os.path.basename(full_input_filename).removesuffix(".brainflow.csv")
                    date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d-%H-%M-%S')
                    if (date_time_obj > datetime(2024, 9, 1, 0, 0, 0)):
                        channels = ['Fpz-M1']
        
                    if channels is not None:
                        log(f"Processing file {full_input_filename} with channels {channels}")
                        # Changing to gap-filling on 30th Nov
                        convert.convert_and_save_brainflow_file_with_gap_filling(log, full_input_filename, full_output_filename, channels)

        except Exception as e:
            msg = "Error processing file: " + full_input_filename
            log(msg)
            log(e)
            errors.append(msg)


2025-01-12 08:45:30.360075 Skipping file C:\dev\play\brainwave-data\2024-10-02-21-20-34.brainflow.csv as C:\dev\play\brainwave-data\2024-10-02-21-20-34\raw.fif and C:\dev\play\brainwave-data\2024-10-02-21-20-34\2024-10-02-21-20-34.brainflow.csv.bz2 already exist
2025-01-12 08:45:30.360075 Skipping file C:\dev\play\brainwave-data\2024-10-06-21-29-18.brainflow.csv as C:\dev\play\brainwave-data\2024-10-06-21-29-18\raw.fif and C:\dev\play\brainwave-data\2024-10-06-21-29-18\2024-10-06-21-29-18.brainflow.csv.bz2 already exist
2025-01-12 08:45:30.362223 Skipping file C:\dev\play\brainwave-data\2024-10-08-21-21-34.brainflow.csv as C:\dev\play\brainwave-data\2024-10-08-21-21-34\raw.fif and C:\dev\play\brainwave-data\2024-10-08-21-21-34\2024-10-08-21-21-34.brainflow.csv.bz2 already exist
2025-01-12 08:45:30.363105 Skipping file C:\dev\play\brainwave-data\2024-10-09-21-22-04.brainflow.csv as C:\dev\play\brainwave-data\2024-10-09-21-22-04\raw.fif and C:\dev\play\brainwave-data\2024-10-09-21-22-04\

C:\dev\play\brainwave-processor\convert.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idx_and_eeg_channels_and_timestamp['datetime'] = pd.to_datetime(idx_and_eeg_channels_and_timestamp['timestamp'],unit="s").dt.tz_localize('UTC')
Processing rows: 100%|██████████| 8501906/8501906 [00:09<00:00, 883416.84it/s] 


Gaps:  datetime
False    8493314
True          39
Name: count, dtype: int64


Processing rows:   0%|          | 42188/8493352 [00:00<01:47, 78531.90it/s]

Gap 47688 start time:  2025-01-11 21:22:40.676285028+00:00
Gap 47688 end time:  2025-01-11 21:23:54.253644943+00:00


...

Gap 6498685 start time:  2025-01-12 04:34:23.418447971+00:00
Gap 6498685 end time:  2025-01-12 04:34:24.438766956+00:00


Processing rows:  88%|████████▊ | 7500903/8493352 [01:36<00:12, 82118.05it/s]

Gap 7488439 start time:  2025-01-12 05:40:16.202073097+00:00
Gap 7488439 end time:  2025-01-12 05:40:17.222178936+00:00


Processing rows: 100%|██████████| 8493352/8493352 [01:49<00:00, 77569.54it/s]
C:\dev\play\brainwave-processor\convert.py:151: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  copied.fillna('', inplace=True)


2025-01-12 09:00:27.795597 Initial timestamp: 2025-01-11 21:19:30.295933008+00:00 from 1736630370.295933
2025-01-12 09:00:27.921755 Info <Info | 7 non-empty values
 bads: []
 ch_names: Fpz-M1
 chs: 1 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: 2025-01-11 21:19:30 UTC
 nchan: 1
 projs: []
 sfreq: 250.0 Hz
>
2025-01-12 09:00:28.613970 Memory Usage: 1852.25 MB GC to 1955.30 MB
2025-01-12 09:00:28.613970 Saving to C:\dev\play\brainwave-data\2025-01-11-21-19-30\raw.fif


In [76]:
errors

['Error processing file: C:\\dev\\play\\brainwave-data\\2025-01-01-21-40-36.brainflow.csv',
 'Error processing file: C:\\dev\\play\\brainwave-data\\2025-01-08-21-40-21.brainflow.csv']

In [77]:
processed

['Processing C:\\dev\\play\\brainwave-data\\2025-01-01-21-40-36.brainflow.csv',
 'Processing C:\\dev\\play\\brainwave-data\\2025-01-08-21-40-21.brainflow.csv',
 'Compressing C:\\dev\\play\\brainwave-data\\2025-01-11-21-19-30.brainflow.csv',
 'Processing C:\\dev\\play\\brainwave-data\\2025-01-11-21-19-30.brainflow.csv']

## Run YASA pipeline on FIF files

In [78]:
import pandas as pd
import run_feature_pipeline
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

errors = []
dataframes = []

# Could get these working later
skip_list = ['2024-07-23-22-40-25', '2024-07-28-22-29-49', '2024-09-18-21-25-08', '2024-09-18-21-28-11', '2024-09-19-21-29-42']
import io
import sys
import contextlib
import traceback
from importlib import reload

reload(run_feature_pipeline)

def process_file(root, dir_name, force=False):
    output_buffer = io.StringIO()
    with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):

        try:
            input_file = os.path.join(root, dir_name, "raw.fif")
            if dir_name in skip_list:
                log(f"Skipping {dir_name}: {input_file}")
                return None, False, output_buffer.getvalue(), "Skipped"

            log(f"Processing file: {input_file}")
            if os.path.exists(input_file):
                yasa_df, cached = run_feature_pipeline.cached_pipeline(log, input_file, force or global_force)
                return yasa_df, cached, output_buffer.getvalue(), True
            else:
                log(f"File not found: {input_file}")
                return None, False, output_buffer.getvalue(), "File not found"

        except Exception as e:
            msg = f"Error processing file: {input_file} - {e}"
            log(msg)
            errors.append(msg)
            return None, False, output_buffer.getvalue(), "Error: " + str(e)

# Testing
# yasa_df, cached, stdout, success_or_msg = process_file("C:\\dev\\play\\brainwave-data", "2024-07-16-23-14-52", False)
# stdout, success_or_msg

In [79]:
results = pd.DataFrame(columns=['dir_name', 'cached', 'stdout', 'success_or_msg'])

from tqdm.auto import tqdm

dirs = next(os.walk(input_dir))[1]
for idx, dir_name in enumerate(tqdm(dirs, desc="YASA pipeline")):
    yasa_df, cached, stdout, success_or_msg = process_file(input_dir, dir_name)
    results.loc[len(results)] = [dir_name, cached, stdout, success_or_msg]

results

YASA pipeline:   0%|          | 0/129 [00:00<?, ?it/s]

12-Jan-25 09:01:36 | WARNING | Hypnogram is SHORTER than data by 9.11 seconds. Padding hypnogram with last value to match data.size.


,dir_name,cached,stdout,success_or_msg
0,08-07-2024--22-51-16,False,2025-01-12 09:00:34.459199 Processing file: C:...,File not found
1,09-07-2024--22-52-25,False,2025-01-12 09:00:34.459199 Processing file: C:...,File not found
2,2024-07-08-22-51-16,True,2025-01-12 09:00:34.459199 Processing file: C:...,True
3,2024-07-11-22-46-18,True,2025-01-12 09:00:34.479480 Processing file: C:...,True
4,2024-07-12-22-38-58,True,2025-01-12 09:00:34.499644 Processing file: C:...,True
...,...,...,...,...
124,2025-01-08-21-40-21,False,2025-01-12 09:00:45.087441 Processing file: C:...,File not found
125,2025-01-08-21-41-09,True,2025-01-12 09:00:45.097423 Processing file: C:...,True
126,2025-01-11-21-19-30,False,2025-01-12 09:00:45.109228 Processing file: C:...,True
127,compressed_files,False,2025-01-12 09:02:31.794883 Processing file: C:...,File not found


# Physiological pipeline

In [80]:
import traceback
import os

from importlib import reload
import run_physical_features_pipeline
reload(run_physical_features_pipeline)
import io
import contextlib
from datetime import datetime, timedelta


def process_file_for_physical(root, dir_name, force=False):
    output_buffer = io.StringIO()
    with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
        input_fif_file = os.path.join(root, dir_name, "raw.fif")
        input_csv_file = os.path.join(root, dir_name, "raw.yasa.csv")
        try:
            if os.path.exists(input_csv_file):
                yasa_df = pd.read_csv(input_csv_file)
                physical_features_df, cached = run_physical_features_pipeline.cached_physical_features_pipeline(log, input_fif_file, yasa_df, force or global_force)
                return physical_features_df, cached, output_buffer.getvalue(), True
            else:
                log(f"File not found: {input_fif_file} or {input_csv_file}")
                return None, False, output_buffer.getvalue(), "File not found"
        except Exception as e:
            return None, False, output_buffer.getvalue(), (str(e) + " " + traceback.format_exc())

# Testing
# physical_features_df, cached, stdout, success_or_msg = process_file_for_physical("C:\\dev\\play\\brainwave-data", "2024-12-11-21-31-37", False)
# display(stdout)
# display(success_or_msg)
# display(physical_features_df)

In [81]:
from tqdm.auto import tqdm

results = pd.DataFrame(columns=['dir_name', 'cached', 'stdout', 'success_or_msg'])

dirs = next(os.walk(input_dir))[1]
for idx, dir_name in enumerate(tqdm(dirs, desc="Physiological pipeline")):
    physical_features_df, cached, stdout, success_or_msg = process_file_for_physical(input_dir, dir_name)
    results.loc[len(results)] = [dir_name, cached, stdout, success_or_msg]

results

Physiological pipeline:   0%|          | 0/129 [00:00<?, ?it/s]

,dir_name,cached,stdout,success_or_msg
0,08-07-2024--22-51-16,False,2025-01-12 09:02:32.295707 File not found: C:\...,File not found
1,09-07-2024--22-52-25,False,2025-01-12 09:02:32.297889 File not found: C:\...,File not found
2,2024-07-08-22-51-16,True,2025-01-12 09:02:32.318450 Loading cached file...,True
3,2024-07-11-22-46-18,True,2025-01-12 09:02:32.331209 Loading cached file...,True
4,2024-07-12-22-38-58,True,2025-01-12 09:02:32.342609 Loading cached file...,True
...,...,...,...,...
124,2025-01-08-21-40-21,False,2025-01-12 09:02:35.020689 File not found: C:\...,File not found
125,2025-01-08-21-41-09,True,2025-01-12 09:02:35.020689 Loading cached file...,True
126,2025-01-11-21-19-30,False,2025-01-12 09:02:35.036429 No cached file C:\d...,True
127,compressed_files,False,2025-01-12 09:02:37.041866 File not found: C:\...,File not found


# Post-YASA pipeline

In [114]:
import traceback
import os

from importlib import reload
import run_post_yasa_pipeline
reload(run_post_yasa_pipeline)
import yasa_features
reload(yasa_features)


def process_file_for_post_yasa(root, dir_name, force=False):
    output_buffer = io.StringIO()
    with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):

        input_fif_file = os.path.join(root, dir_name, "raw.fif")
        input_csv_file = os.path.join(root, dir_name, "raw.yasa.csv")
        try:
            log(f"Processing file: " + input_fif_file)
            if os.path.exists(input_fif_file) and os.path.exists(input_csv_file):
                yasa_df = pd.read_csv(input_csv_file)
                post_yasa_df, cached = run_post_yasa_pipeline.cached_post_yasa_pipeline(log, input_fif_file, yasa_df, stats_df, force or global_force)
                return post_yasa_df, cached, output_buffer.getvalue(), True
            else:
                log(f"File not found: {input_fif_file} or {input_csv_file}")
                return None, False, output_buffer.getvalue(), "File not found"
        except Exception as e:
            return None, False, output_buffer.getvalue(), e

# Testing
post_yasa_df, cached, stdout, success_or_msg = process_file_for_post_yasa("C:\\dev\\play\\brainwave-data", "2025-01-11-21-19-30", False)
display(stdout)
display(success_or_msg)

"2025-01-12 09:54:22.974797 Processing file: C:\\dev\\play\\brainwave-data\\2025-01-11-21-19-30\\raw.fif\n2025-01-12 09:54:22.979078 No cached file C:\\dev\\play\\brainwave-data\\2025-01-11-21-19-30\\raw.post_yasa.csv, rebuilding\n2025-01-12 09:54:22.979078 Loading MNE file C:\\dev\\play\\brainwave-data\\2025-01-11-21-19-30\\raw.fif\n2025-01-12 09:54:22.979078 Reading file C:\\dev\\play\\brainwave-data\\2025-01-11-21-19-30\\raw.fif\n2025-01-12 09:54:23.525312 Finished reading file C:\\dev\\play\\brainwave-data\\2025-01-11-21-19-30\\raw.fif\n2025-01-12 09:54:25.302153 Memory Usage: 1372.61 MB GC to 1372.61 MB\n2025-01-12 09:54:25.302153 Extracting YASA features\nC:\\dev\\play\\brainwave-processor\\yasa_features.py:98: RuntimeWarning: Iterative multi-taper PSD computation did not converge.\n  psd, freqs = psd_array_multitaper(epochs, sf, adaptive=True, normalization='full', verbose=0)\nC:\\dev\\play\\brainwave-processor\\yasa_features.py:98: RuntimeWarning: Iterative multi-taper PSD comp

True

In [115]:
results = pd.DataFrame(columns=['dir_name', 'cached', 'stdout', 'success_or_msg'])

all_post_yasa_dfs = []

dirs = next(os.walk(input_dir))[1]
for idx, dir_name in enumerate(tqdm(dirs, desc="Post YASA pipeline")):
    post_yasa_df, cached, stdout, success_or_msg = process_file_for_post_yasa(input_dir, dir_name)
    all_post_yasa_dfs.append(post_yasa_df)
    results.loc[len(results)] = [dir_name, cached, stdout, success_or_msg]
    # error_count = len(results[results['success_or_msg'] != True])
    # if (error_count > 5):
    #     log(f"Stopping due to {error_count} errors")
    #     break

results


Post YASA pipeline:   0%|          | 0/129 [00:00<?, ?it/s]

,dir_name,cached,stdout,success_or_msg
0,08-07-2024--22-51-16,False,2025-01-12 09:54:55.851514 Processing file: C:...,File not found
1,09-07-2024--22-52-25,False,2025-01-12 09:54:55.852514 Processing file: C:...,File not found
2,2024-07-08-22-51-16,True,2025-01-12 09:54:55.855516 Processing file: C:...,True
3,2024-07-11-22-46-18,True,2025-01-12 09:54:55.921825 Processing file: C:...,True
4,2024-07-12-22-38-58,True,2025-01-12 09:54:56.073385 Processing file: C:...,True
...,...,...,...,...
124,2025-01-08-21-40-21,False,2025-01-12 09:55:05.934813 Processing file: C:...,File not found
125,2025-01-08-21-41-09,False,2025-01-12 09:55:05.936813 Processing file: C:...,True
126,2025-01-11-21-19-30,True,2025-01-12 09:55:19.943848 Processing file: C:...,True
127,compressed_files,False,2025-01-12 09:55:20.035177 Processing file: C:...,File not found


# Recalculate scalings
N.b. can be run frequently but will only be picked up by new runs.  Maybe worth occasionally regenerating all old files.
And yes, for new features have to rerun the pipeline on everything, then generate the stats here, then rerun the pipeline again on everything to have them use those.

In [84]:
# post_yasa_dfs_combined = pd.concat(all_post_yasa_dfs)

In [85]:
# from scaling import only_eeg
#
# only_eeg_cols = list(only_eeg(post_yasa_dfs_combined).columns)
# assert any(col.startswith("Main") for col in only_eeg_cols), "No column starting with 'Main' found in only_eeg_cols"

In [86]:
# #Expensive to do every time now
# import scaling
# reload(scaling)
#
# stats = scaling.stats(post_yasa_dfs_combined)
# stats.to_csv(input_dir + "/stats.csv")
# # assert any(stats['Column'].str.startswith("Main")), "No row starting with 'Main' found in column_name"
# stats

,Column,P10,P90,Min,Max,W_P10,W_P90,W_Min,W_Max,N1_P10,...,N3_Min,N3_Max,Sleep_P10,Sleep_P90,Sleep_Min,Sleep_Max,NonDeepSleep_P10,NonDeepSleep_P90,NonDeepSleep_Min,NonDeepSleep_Max
0,F8-M1_eeg_abspow,0.047727,0.951658,8.534915e-26,776964.700000,0.097950,4.817721,8.534915e-26,776964.700000,0.039776,...,0.073082,21.204674,0.045006,0.642558,0.020684,85.312390,0.046052,0.882197,8.534915e-26,776964.700000
1,F8-M1_eeg_abspow_c7min_norm,-0.089688,0.567241,-2.093740e-01,47832.670000,-0.010419,1.219719,-2.085501e-01,47832.670000,-0.098471,...,-0.209374,10.396397,-0.099290,0.343220,-0.209374,10631.603000,-0.095002,0.515971,-2.085501e-01,47832.670000
2,F8-M1_eeg_abspow_p2min_norm,-0.068377,0.557725,-2.503299e-01,79046.375000,-0.009382,1.490748,-2.443668e-01,79046.375000,-0.076661,...,-0.240798,1.948651,-0.077876,0.335586,-0.250330,186.977980,-0.073150,0.478114,-2.503299e-01,79046.375000
3,F8-M1_eeg_alpha,0.012441,0.125355,8.034852e-05,0.398397,0.004978,0.112786,8.034852e-05,0.398397,0.064327,...,0.000273,0.106273,0.015318,0.126659,0.000133,0.307049,0.017877,0.129320,8.034852e-05,0.398397
4,F8-M1_eeg_alpha_c7min_norm,-0.519022,0.378819,-7.588421e-01,1.428529,-0.518486,0.307230,-7.278957e-01,1.428529,-0.056613,...,-0.758842,0.811080,-0.520056,0.392001,-0.758842,1.120482,-0.370760,0.404161,-7.278957e-01,1.428529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,F4-M1_eeg_thetaabsaa_c7min_norm,-0.278986,0.404271,-4.454707e-01,12827.550000,-0.087921,1.126729,-3.684435e-01,12827.550000,-0.266314,...,-0.047119,0.452393,-0.306157,0.185360,-0.445471,0.678620,-0.290226,0.433802,-4.454707e-01,12827.550000
842,F4-M1_eeg_thetaabsaa_p2min_norm,-0.229213,0.353358,-3.893398e-01,16487.691000,-0.071362,1.068929,-2.843512e-01,16487.691000,-0.239686,...,-0.050917,0.541523,-0.258175,0.157967,-0.389340,1.346127,-0.243130,0.398072,-3.893398e-01,16487.691000
843,F4-M1_eeg_thetaabsab,0.046761,0.903548,1.705335e-02,385554.940000,0.052995,2.566446,1.705335e-02,385554.940000,0.034514,...,0.253479,8.216130,0.045584,0.776523,0.023029,8.216130,0.044918,0.420556,1.705335e-02,385554.940000
844,F4-M1_eeg_thetaabsab_c7min_norm,-0.087463,0.599673,-1.146235e-01,6310.061000,-0.018629,0.989809,-1.058638e-01,6310.061000,-0.103516,...,0.006596,1.428683,-0.094318,0.471820,-0.114624,5.504155,-0.090948,0.439211,-1.146235e-01,6310.061000


# Microwakings

In [87]:
from models.microwakings_1 import microwakings_pipeline
import os

import io

from importlib import reload
reload(microwakings_pipeline)

def process_microwakings(root, dir_name, force=False):
    output_buffer = io.StringIO()
    with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
        input_fif_file = os.path.join(root, dir_name, "raw.fif")
        input_csv_file = os.path.join(root, dir_name, "raw.post_yasa.csv")
        try:
            log(f"Processing file: " + input_fif_file)
            tqdm.write(f"Processing file: " + input_fif_file)
            if not os.path.exists(input_fif_file):
                return None, False, output_buffer.getvalue(), "File not found " + input_fif_file
            if not os.path.exists(input_csv_file):
                return None, False, output_buffer.getvalue(), "File not found " + input_csv_file

            post_yasa_df = pd.read_csv(input_csv_file)
            microwakings_df, cached = microwakings_pipeline.cached_microwakings_pipeline(log, input_fif_file, post_yasa_df, force or global_force)
            return microwakings_df, cached, output_buffer.getvalue(), True
        except Exception as e:
            return None, False, output_buffer.getvalue(), "Error: " + str(e)

# Testing
# microwakings_df, cached, stdout, success_or_msg = process_microwakings("C:\\dev\\play\\brainwave-data", "2024-11-29-21-18-33", False)
# stdout, success_or_msg

In [117]:
results = pd.DataFrame(columns=['dir_name', 'cached', 'stdout', 'success_or_msg'])

dirs = next(os.walk(input_dir))[1]
for idx, dir_name in enumerate(tqdm(dirs, desc="Microwakings pipeline")):
    # print(dir_name)
    microwakings_df, cached, stdout, success_or_msg = process_microwakings(input_dir, dir_name)
    results.loc[len(results)] = [dir_name, cached, stdout, success_or_msg]

results

Microwakings pipeline:   0%|          | 0/129 [00:00<?, ?it/s]

,dir_name,cached,stdout,success_or_msg
0,08-07-2024--22-51-16,False,2025-01-12 09:58:01.358795 Processing file: C:...,File not found C:\dev\play\brainwave-data\08-0...
1,09-07-2024--22-52-25,False,2025-01-12 09:58:01.359795 Processing file: C:...,File not found C:\dev\play\brainwave-data\09-0...
2,2024-07-08-22-51-16,True,2025-01-12 09:58:01.361794 Processing file: C:...,True
3,2024-07-11-22-46-18,True,2025-01-12 09:58:01.420730 Processing file: C:...,True
4,2024-07-12-22-38-58,True,2025-01-12 09:58:01.552004 Processing file: C:...,True
...,...,...,...,...
124,2025-01-08-21-40-21,False,2025-01-12 09:58:11.046584 Processing file: C:...,File not found C:\dev\play\brainwave-data\2025...
125,2025-01-08-21-41-09,True,2025-01-12 09:58:11.047546 Processing file: C:...,True
126,2025-01-11-21-19-30,True,2025-01-12 09:58:11.111292 Processing file: C:...,True
127,compressed_files,False,2025-01-12 09:58:11.190941 Processing file: C:...,File not found C:\dev\play\brainwave-data\comp...


# Post-human pipeline
This requires user interaction first to provide sleep times etc.

In [89]:
from sleep_events import load_days_data
from sleep_events import pimp_my_days_data

days_data = load_days_data(True)

In [90]:
from models.eeg_states.eeg_states import load_and_prepare_settling_eeg_state_events

tired_wired_eeg_state_events = load_and_prepare_settling_eeg_state_events()

C:\Users\graha\.conda\envs\tf\lib\site-packages\google\cloud\firestore_v1\base_collection.py:300: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:130: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-10-09 20:17:27.992000+00:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  events.at[i, 'since'] = since_time
C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:132: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  events['duration'] = events['timestamp'] - events['since']


In [91]:
import run_post_human_pipeline
reload(run_post_human_pipeline)
from run_post_human_pipeline import cached_post_human_pipeline

def process_file_for_post_human(root, dir_name, force=False):
    output_buffer = io.StringIO()
    with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):

        input_fif_file = os.path.join(root, dir_name, "raw.fif")
        input_csv_file = os.path.join(root, dir_name, "raw.post_yasa.csv")
        try:
            log(f"Processing file: " + input_fif_file)
            if not os.path.exists(input_fif_file):
                return None, False, output_buffer.getvalue(), "File not found " + input_fif_file
            if not os.path.exists(input_csv_file):
                return None, False, output_buffer.getvalue(), "File not found " + input_csv_file

            post_yasa_df = pd.read_csv(input_csv_file)
            post_human_df, cached = cached_post_human_pipeline(log, dir_name, input_fif_file, stats_df, days_data, post_yasa_df, tired_wired_eeg_state_events, force or global_force)
            return post_human_df, cached, output_buffer.getvalue(), True
        except Exception as e:
            return None, False, output_buffer.getvalue(), "Error: " + str(e)

# Testing
# post_human_df, cached, stdout, success_or_msg = process_file_for_post_human("C:\\dev\\play\\brainwave-data", "2024-07-22-22-24-06", True)
# stdout, success_or_msg

In [118]:
from run_post_human_pipeline import cached_post_human_pipeline

results = pd.DataFrame(columns=['dir_name', 'cached', 'stdout', 'success_or_msg'])

dirs = next(os.walk(input_dir))[1]
for idx, dir_name in enumerate(tqdm(dirs, desc="Post human pipeline")):
    post_human_df, cached, stdout, success_or_msg = process_file_for_post_human(input_dir, dir_name)
    results.loc[len(results)] = [dir_name, cached, stdout, success_or_msg]

results

Post human pipeline:   0%|          | 0/129 [00:00<?, ?it/s]

,dir_name,cached,stdout,success_or_msg
0,08-07-2024--22-51-16,False,2025-01-12 09:58:17.376514 Processing file: C:...,File not found C:\dev\play\brainwave-data\08-0...
1,09-07-2024--22-52-25,False,2025-01-12 09:58:17.377546 Processing file: C:...,File not found C:\dev\play\brainwave-data\09-0...
2,2024-07-08-22-51-16,True,2025-01-12 09:58:17.380514 Processing file: C:...,True
3,2024-07-11-22-46-18,True,2025-01-12 09:58:17.512851 Processing file: C:...,True
4,2024-07-12-22-38-58,True,2025-01-12 09:58:17.802913 Processing file: C:...,True
...,...,...,...,...
124,2025-01-08-21-40-21,False,2025-01-12 09:58:38.085400 Processing file: C:...,File not found C:\dev\play\brainwave-data\2025...
125,2025-01-08-21-41-09,False,2025-01-12 09:58:38.086415 Processing file: C:...,True
126,2025-01-11-21-19-30,False,2025-01-12 09:58:40.120849 Processing file: C:...,True
127,compressed_files,False,2025-01-12 09:58:42.861140 Processing file: C:...,File not found C:\dev\play\brainwave-data\comp...


# Stitch all pipelines together

In [119]:
from stitch_pipeline_results import stitch_day_optimised


def process_pipeline_stitch(root, dir_name, force=False):
    output_buffer = io.StringIO()
    with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):

        post_yasa_file = os.path.join(root, dir_name, "raw.post_yasa.csv")
        post_human_file = os.path.join(root, dir_name, "raw.post_human.csv")

        try:
            if os.path.exists(post_yasa_file):
                # Multiple consumers expect this file
                with_features = os.path.join(root, dir_name, "raw.with_features.csv")
                if not os.path.exists(with_features) or force:
                    stitched = stitch_day_optimised(input_dir, dir_name, force or global_force)
                    assert("Main_eeg_sigmaabs" in stitched.columns), "Main_eeg_sigmaabs not found in stitched columns"
                    stitched.to_csv(with_features, index=False)

            return output_buffer.getvalue(), True
        except Exception as e:
            return output_buffer.getvalue(), "Error: " + str(e)


# Testing
stdout, success_or_msg = process_pipeline_stitch("C:\\dev\\play\\brainwave-data", "2024-12-31-21-12-28", True)
stdout, success_or_msg


('C:\\dev\\play\\brainwave-processor\\stitch_pipeline_results.py:189: DtypeWarning: Columns (590,591) have mixed types. Specify dtype option on import or set low_memory=False.\n  post_human_df = pd.read_csv(post_human_file)\n',
 True)

In [120]:
results = pd.DataFrame(columns=['dir_name', 'stdout', 'success_or_msg'])

dirs = next(os.walk(input_dir))[1]
for idx, dir_name in enumerate(tqdm(dirs, desc="Stitching pipelines")):
    stdout, success_or_msg = process_pipeline_stitch(input_dir, dir_name, False)
    results.loc[len(results)] = [dir_name, stdout, success_or_msg]

results

Stitching pipelines:   0%|          | 0/129 [00:00<?, ?it/s]

,dir_name,stdout,success_or_msg
0,08-07-2024--22-51-16,,True
1,09-07-2024--22-52-25,,True
2,2024-07-08-22-51-16,,True
3,2024-07-11-22-46-18,,True
4,2024-07-12-22-38-58,,True
...,...,...,...
124,2025-01-08-21-40-21,,True
125,2025-01-08-21-41-09,,True
126,2025-01-11-21-19-30,C:\dev\play\brainwave-processor\stitch_pipelin...,True
127,compressed_files,,True


# Analyse data completeness

In [95]:
# import os
# import pandas as pd
# from datetime import datetime
#
# dfs = []
# modification_times = []
#
# for root, dirs, files in os.walk(input_dir):
#     for idx, dir_name in enumerate(dirs):
#         input_file = os.path.join(root, dir_name, "raw.post_human.csv")
#         if os.path.exists(input_file):
#             df = pd.read_csv(input_file)
#             dfs.append(df)
#
#             # Get the last modification time of the file
#             mod_time = os.path.getmtime(input_file)
#             mod_time_dt = datetime.fromtimestamp(mod_time)
#             modification_times.append(mod_time_dt)
#
# # Concatenate all dataframes
# yasa_df = pd.concat(dfs, ignore_index=True)
#
# # Calculate the number of days from today for each modification time
# today = datetime.now()
# days_since_modification = [(today - mod_time).days for mod_time in modification_times]
#
# # Create a DataFrame with the modification times and days since modification
# modification_df = pd.DataFrame({
#     'file': [os.path.join(root, dir_name, "raw.post_human.csv") for root, dirs, files in os.walk(input_dir) for dir_name in dirs if os.path.exists(os.path.join(root, dir_name, "raw.post_human.csv"))],
#     'modification_time': modification_times,
#     'days_since_modification': days_since_modification
# })
#
# modification_df

# Upload to GCS

In [96]:
# Test
import upload
reload(upload)
from upload import upload_dir_to_gcs_skipping_existing

# Test
# upload_dir_to_gcs_skipping_existing(lambda x: tqdm.write(x), 'examined-life-derived-eeg', "C:\\dev\\play\\brainwave-data\\2024-12-18-21-32-18", "2024-12-18-21-32-18")


In [121]:
from upload import upload_dir_to_gcs_skipping_existing
import os

errors = []
dataframes = []

from tqdm.auto import tqdm

dirs = next(os.walk(input_dir))[1]
for idx, dir_name in enumerate(tqdm(dirs)):
# for root, dirs, files in os.walk(input_dir):
#     for dir_name in reversed(dirs):
    input_file = os.path.join(input_dir, dir_name, "raw.fif")
    full_dir_name = os.path.join(input_dir, dir_name)
    try:
        upload_dir_to_gcs_skipping_existing(lambda x: tqdm.write(x), 'examined-life-derived-eeg', full_dir_name, dir_name)
    except Exception as e:
        log("Error processing file: " + input_dir)
        errors.append("Error processing file: " + input_file + " - " + str(e))
        log(e)

for error in errors:
    log(error)

log("All uploaded")

  0%|          | 0/129 [00:00<?, ?it/s]

File C:\dev\play\brainwave-data\08-07-2024--22-51-16\08-07-2024--22-51-16.brainflow.csv.bz2 modified at 2024-09-25 18:43:53.067379
File C:\dev\play\brainwave-data\08-07-2024--22-51-16\directory_checksum.md5 modified at 2024-12-21 08:41:20.733564
Skipping upload of C:\dev\play\brainwave-data\08-07-2024--22-51-16, no files modified since last checksum.
File C:\dev\play\brainwave-data\09-07-2024--22-52-25\09-07-2024--22-52-25.brainflow.csv.bz2 modified at 2024-09-25 18:45:02.372116
File C:\dev\play\brainwave-data\09-07-2024--22-52-25\directory_checksum.md5 modified at 2024-12-21 08:41:22.898321
Skipping upload of C:\dev\play\brainwave-data\09-07-2024--22-52-25, no files modified since last checksum.
File C:\dev\play\brainwave-data\2024-07-08-22-51-16\directory_checksum.md5 modified at 2025-01-11 09:02:54.253537
File C:\dev\play\brainwave-data\2024-07-08-22-51-16\raw.average_slow_wave.png modified at 2025-01-10 17:37:20.299037
File C:\dev\play\brainwave-data\2024-07-08-22-51-16\raw.edf mod

# Check if can delete Brainwave files that are safely backed up

In [98]:

import os

errors = []
can_delete = []
cannot_delete = []

for root, dirs, files in os.walk(input_dir):
    for idx, file_name in enumerate(files):
        full_input_filename = os.path.join(root, file_name)
        if full_input_filename.endswith(".brainflow.csv"):
            compressed_full_output_filename = get_brainflow_compressed_filename(full_input_filename)
                
            if os.path.exists(compressed_full_output_filename):
                can_delete.append({
                    'backed_up': compressed_full_output_filename,
                    'full_filename': full_input_filename
                })
            else:
                cannot_delete.append(full_input_filename)
    

In [99]:
can_delete_df = pd.DataFrame(can_delete)
can_delete_df

,backed_up,full_filename
0,C:\dev\play\brainwave-data\2024-10-02-21-20-34...,C:\dev\play\brainwave-data\2024-10-02-21-20-34...
1,C:\dev\play\brainwave-data\2024-10-06-21-29-18...,C:\dev\play\brainwave-data\2024-10-06-21-29-18...
2,C:\dev\play\brainwave-data\2024-10-08-21-21-34...,C:\dev\play\brainwave-data\2024-10-08-21-21-34...
3,C:\dev\play\brainwave-data\2024-10-09-21-22-04...,C:\dev\play\brainwave-data\2024-10-09-21-22-04...
4,C:\dev\play\brainwave-data\2024-10-11-21-16-27...,C:\dev\play\brainwave-data\2024-10-11-21-16-27...
...,...,...
64,C:\dev\play\brainwave-data\2025-01-08-21-38-53...,C:\dev\play\brainwave-data\2025-01-08-21-38-53...
65,C:\dev\play\brainwave-data\2025-01-08-21-39-36...,C:\dev\play\brainwave-data\2025-01-08-21-39-36...
66,C:\dev\play\brainwave-data\2025-01-08-21-40-21...,C:\dev\play\brainwave-data\2025-01-08-21-40-21...
67,C:\dev\play\brainwave-data\2025-01-08-21-41-09...,C:\dev\play\brainwave-data\2025-01-08-21-41-09...


In [100]:
cannot_delete

[]

In [101]:
can_delete_df['full_filename']

0     C:\dev\play\brainwave-data\2024-10-02-21-20-34...
1     C:\dev\play\brainwave-data\2024-10-06-21-29-18...
2     C:\dev\play\brainwave-data\2024-10-08-21-21-34...
3     C:\dev\play\brainwave-data\2024-10-09-21-22-04...
4     C:\dev\play\brainwave-data\2024-10-11-21-16-27...
                            ...                        
64    C:\dev\play\brainwave-data\2025-01-08-21-38-53...
65    C:\dev\play\brainwave-data\2025-01-08-21-39-36...
66    C:\dev\play\brainwave-data\2025-01-08-21-40-21...
67    C:\dev\play\brainwave-data\2025-01-08-21-41-09...
68    C:\dev\play\brainwave-data\2025-01-11-21-19-30...
Name: full_filename, Length: 69, dtype: object

# Check if can delete Cyton files that are safely backed up

In [102]:
import webserver
import bz2
import shutil
import os
import time


errors = []
can_delete = []
cannot_delete = []

for root, dirs, files in os.walk(input_dir):
    for idx, file_name in enumerate(files):
        full_input_filename = os.path.join(root, file_name)
        if file_name.startswith("OBCI_") and file_name.endswith(".TXT.bz2"):
            can_delete.append({
                'file_name': file_name.removesuffix(".bz2"),
                'full_filename': full_input_filename
            })

In [103]:
can_delete

[{'file_name': 'OBCI_0B.TXT',
  'full_filename': 'C:\\dev\\play\\brainwave-data\\2024-09-24-22-43-11\\OBCI_0B.TXT.bz2'},
 {'file_name': 'OBCI_0D.TXT',
  'full_filename': 'C:\\dev\\play\\brainwave-data\\2024-09-25-21-57-01\\OBCI_0D.TXT.bz2'},
 {'file_name': 'OBCI_1F.TXT',
  'full_filename': 'C:\\dev\\play\\brainwave-data\\2024-10-01-21-36-11\\OBCI_1F.TXT.bz2'},
 {'file_name': 'OBCI_22.TXT',
  'full_filename': 'C:\\dev\\play\\brainwave-data\\2024-10-04-21-13-39\\OBCI_22.TXT.bz2'},
 {'file_name': 'OBCI_23.TXT',
  'full_filename': 'C:\\dev\\play\\brainwave-data\\2024-10-05-21-09-46\\OBCI_23.TXT.bz2'},
 {'file_name': 'OBCI_F7.TXT',
  'full_filename': 'C:\\dev\\play\\brainwave-data\\2024-10-21-21-30-18\\OBCI_F7.TXT.bz2'},
 {'file_name': 'OBCI_03.TXT',
  'full_filename': 'C:\\dev\\play\\brainwave-data\\2024-10-23-21-24-02\\OBCI_03.TXT.bz2'},
 {'file_name': 'OBCI_08.TXT',
  'full_filename': 'C:\\dev\\play\\brainwave-data\\compressed_files\\OBCI_08.TXT.bz2'}]

In [104]:
cyton_file_locations = [input_dir, "d:", "e:", "x:"]

for cyton_file_location in cyton_file_locations:
    for root, dirs, files in os.walk(cyton_file_location):
        for idx, file in enumerate(files):
            matching_record = next((f for f in can_delete if f['file_name'] == file), None)
            if matching_record:
                full_filename = os.path.join(root, file)
                log(f"Could delete {full_filename} as backed up in {matching_record['full_filename']}")

2025-01-12 09:29:40.062228 Could delete C:\dev\play\brainwave-data\OBCI_F7.TXT as backed up in C:\dev\play\brainwave-data\2024-10-21-21-30-18\OBCI_F7.TXT.bz2


In [105]:
import memory

memory.garbage_collect(log)

2025-01-12 09:29:42.289054 Memory Usage: 2263.59 MB GC to 2262.93 MB


In [106]:
import ctypes

ctypes.windll.user32.MessageBoxW(0, "Compression is complete!", "Alert", 0x40 | 0x1)

1